In [ ]:
from __future__ import print_function
import numpy as np
import random as rd
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix  
from scipy.optimize import linprog
from sklearn.preprocessing import normalize
from scipy.spatial.distance import pdist, squareform
from scipy.ndimage import gaussian_filter
import scipy as scip
from ortools.linear_solver import pywraplp
from sklearn import preprocessing
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import metrics
from pulp import *
from numpy.random import rand
import matplotlib.pyplot as plt
import cvxpy as cp
import time
from sklearn.model_selection import KFold


import gurobipy as gp
from gurobipy import GRB

# Private Algorithm

In [ ]:
"""Bregman projection"""
def bregman(weights, s, c):
    B = [1] * len(weights)
    for i in range(len(weights)):
        B[i] = min(1, c * weights[i])/s
    return B

In [ ]:
"""The multiplicative weights update algorithm"""
def MWUA(objects, A, b, c, reward, numRounds, eta, optimalValue, rho, alpha, weights, s, c_bregman):
#     weights = rand(len(objects))
    cumulativeReward = 0
    outcomes = []
    for t in range(numRounds):
        dense_weights = bregman(weights, s, c_bregman)
        outcome = oracle(dense_weights, A, b, c, optimalValue, rho, alpha, s)
        outcomes.append(outcome)
        for i in range(len(weights)):
            weights[i] *= np.exp(- eta * reward(i, outcome))[0]
    return weights, outcomes

In [ ]:
class InfeasibleException(Exception):
    pass

In [ ]:
"""Exponential mechanism"""
def oracle(weights, A, b, c, optimalValue, rho, alpha, s):
    weightedVector = A.transpose().dot(weights)
    n, m = A.shape
    variable_size = int((m - 1)/3)
    sensitivity = 3*np.absolute(optimalValue)/(np.min(np.absolute(c[c!=0]))*s)   
    prob = [0] * variable_size * 2
    all_solutions = np.zeros((variable_size * 2 ,m))
    print('sensitivity ' + str(sensitivity))
    for resp in range(variable_size * 2):
        fixed_solution = [0] * variable_size * 2
        fixed_solution[resp] = optimalValue/c[resp + variable_size + 1]
        x = pulp.LpVariable.dicts("x", range(variable_size + 1), 0, None)
        solver = pulp.LpProblem("test problem",pulp.LpMinimize)
        A_for_variable = A[:, :(variable_size + 1)]
        A_for_fixed_solution = A[:, (variable_size + 1):]

        for i in range(n):
            solver += (pulp.lpSum([A_for_variable[i,j] * x[j] for j in range(variable_size + 1)]) + A_for_fixed_solution[i].T.dot(fixed_solution) - b[i] <= rho)
            solver += (pulp.lpSum([A_for_variable[i,j] * x[j] for j in range(variable_size + 1)]) + A_for_fixed_solution[i].T.dot(fixed_solution) - b[i] >= -rho)    

        solver += pulp.lpSum([weightedVector[i] * x[i] for i in range(variable_size + 1)]) 
        solver.solve()
        for i in range(variable_size + 1):
            all_solutions[resp,i] = x[i].varValue
        all_solutions[resp, resp + variable_size + 1] = optimalValue/c[resp + variable_size + 1]
        score = weightedVector.T.dot(all_solutions[resp]) - np.sum(b)
        prob[resp] = np.exp(epsilon * score / (2*sensitivity) - np.floor((epsilon * score / 2*sensitivity)/10) * 10)
    prob = prob/np.sum(prob)
    
    
    p = np.random.choice(np.array(range(variable_size * 2)), 1, p = prob)
    selected = np.reshape(np.array(all_solutions[p]), (m,1))
    return selected

In [ ]:
"""Get rho value"""
def get_rho(A,b,c,OPT):
    A_max = np.amax(np.absolute(A))
    c = c[c != 0]
    c_min = np.amin(np.absolute(c))
    b_max = np.amin(np.absolute(b))
    print('A min', A_max)
    print('c max',c_min)
    rho = (A_max / c_min) * OPT - b_max
    print('rho', np.absolute(rho))
    return np.absolute(rho)

In [ ]:
"""Resolve private LP using the optimal value of this LP"""
def solveGivenOptimalValue(A, b, linearObjective, optimalValue, numRounds, rho, alpha, weights, s, c_bregman):
    m, n = A.shape  # m equations, n variables
    eta = np.sqrt(np.log(m)/numRounds)
    def reward(i, outcome):
        constraint = A[i]
        threshold = b[i]
        return (threshold - np.dot(constraint, outcome)) / (2 * rho) + 1/2
    weights, outcomes = MWUA(
        range(m), A, b, linearObjective, reward, numRounds, eta, optimalValue, rho, alpha, weights, s, c_bregman
    )
    averageVector = np.mean(outcomes, axis = 0) 

    return averageVector

In [ ]:
"""Rewrite input to fit the form of LP feasibility algorithm
c'x s.t. Ax <= b"""

#Correct
#checked by simple example
def get_input(variables, label, nu = 2):
    e = pd.Series([1] * variables.shape[0])
    d = variables.shape[1]
    kernel = rbf_kernel(variables, gamma = 1 / (d * variables.var()))
    D=np.diag(label.flatten())
    A=np.dot(np.dot(D,kernel),D) 
    m,d=A.shape
    I=np.identity(m)
    O_m1=np.reshape(np.repeat(0,m),(m,1))
    O_mm=np.zeros((m,m))
    e=np.reshape(np.repeat(1,m),(m,1))
    append=np.reshape(np.dot(D,e),(m,1))
    A=np.concatenate((A,-append,I,O_mm),axis=1)
    robust=np.zeros((3*m,3*m+1))
    robust[0:m,0:m]=-I
    robust[0:m,(2*m+1):(3*m+1)]=I
    
    robust[m:2*m,0:m]=I
    robust[m:2*m,(2*m+1):(3*m+1)]=I   # delete minus
    
    robust[2*m:3*m,(m+1):(2*m+1)]=I
    A=np.concatenate((A,robust),axis=0)
    A = -A

    b=np.zeros((4*m,1))
    b[0:m,:]=-e
    
    c=np.zeros((3*m+1,1))
    c[(m+1):(2*m+1),:]=-nu*e
    c[(2*m+1):(3*m+1),:]=-e
    return A,b,c,A.shape[0],A.shape[1],label,kernel,D

In [ ]:
"""get otp for original SVM mathematical programming"""
#Correct
#checked by simple example
def get_optimal(A,b,c):
    n,m = A.shape
#     solver = pywraplp.Solver('simple_mip_program',
#                              pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
#     infinity = solver.infinity()
#     objective = solver.Objective()
    x = cp.Variable(m)
    obj = cp.Maximize(cp.sum(x*c))
    print('Add constraints')
    constraints = [A*x <= b.flatten()] 
    print('optimization')
    prob = cp.Problem(obj, constraints)
    prob.solve()
    s = x.value
    print('Objective value =', prob.value)
    solution = np.zeros((A.shape[1],1))
    for i in range(A.shape[1]):
        solution[i,:] = s[i]
    return prob.value, solution

In [ ]:
def testModel(X, X_train, y, u, gamma, D):
    kernel = rbf_kernel(X, Y=X_train, gamma=1)
    ypred = np.dot(kernel, D).dot(u) - gamma
    ypred[ypred <= 0] = -1
    ypred[ypred >= 0] = 1
    pred_error = 1 - np.abs(metrics.accuracy_score(y.flatten(), ypred))
    return pred_error, ypred

# Solve Private LP SVM

In [ ]:
def solve(X, y, alpha_parameters):
    feature_size = X.shape[1]    
    kf = KFold(n_splits=4)
    errors_priv, errors_nonpriv = [], []
    for train_index, test_index in kf.split(X):
        X_train, y_train = X[train_index], y[train_index]
        X_test, y_test = X[test_index], y[test_index]
        train_size = X_train.shape[0]
        A,b,c,m,d,true_label,kernel,D=get_input(X_train, y_train)
        OPT, solution_nonpriv = get_optimal(A,b,c)
        m,d = A.shape
        rho = get_rho(A,b,c,OPT)
        beta = 0.2
        weights = rand(len(b))
        c_bregman = 1
        s = np.sum([min(c_bregman * weight, 1) for weight in weights])
        
        alpha = alpha_parameter * rho
        print('Alpha is ' + str(alpha))
        T = int(np.ceil(36*rho**2*np.log(m)/(alpha**2)))
        print('T is ' + str(T))

        solution_priv = solveGivenOptimalValue(A, b, c, OPT, T, rho, alpha, weights, s, c_bregman)
        u_nonpriv = solution_nonpriv[:train_size]
        gamma_nonpriv = solution_nonpriv[train_size]
        u_priv = solution_priv[:train_size]
        gamma_priv = solution_priv[train_size]
        y_priv = solution_priv[(train_size + 1): (2 * train_size + 1)]
        s_priv = solution_priv[(2 * train_size + 1):]

        error_priv, ypred_priv = testModel(X_test, X_train, y_test, u_priv, gamma_priv, D)
        error_nonpriv, ypred_nonpriv = testModel(X_test, X_train, y_test, u_nonpriv, gamma_nonpriv, D)
        errors_priv.append(error_priv)
        errors_nonpriv.append(error_nonpriv)
    return np.mean(errors_priv), np.mean(errors_nonpriv)

# Adult Data Set

In [ ]:
adult = pd.read_csv('adult_data.csv',sep = ';')
X = adult[adult.columns[0:adult.shape[1]-1]]
cols = X.columns

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)
X = pd.DataFrame(np_scaled, columns = cols)
X.iloc[:,:] = preprocessing.Normalizer(copy = True, norm='l2').fit_transform(X)
X = X.values

y = adult[adult.columns[adult.shape[1]-1:adult.shape[1]]]
y = preprocessing.normalize(y, norm='l2')
X = X[:600]
y = y[:600]
X, y = shuffle(X, y, random_state = 0)
print(X.shape, y.shape)

# KDDCup99 Data Set

In [ ]:
kddcup99 = pd.read_csv('kddcup99.csv', sep = ';')
n,d = kddcup99.shape
y = np.reshape(np.array(kddcup99['label']),(n, 1))
X = kddcup99.drop(columns = ['label'])
cols = X.columns

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)
X = pd.DataFrame(np_scaled, columns = cols)
X.iloc[:,:] = preprocessing.Normalizer(copy = True, norm='l2').fit_transform(X)
X = X.values

y = preprocessing.normalize(y, norm='l2')
X = X[:120]
y = y[:120]
# X, y = shuffle(X, y, random_state = 0)
print(X.shape, y.shape)

# Private Solve

In [ ]:
# alpha_parameter = [5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9]
alpha_parameter = 9
epsilon = 0.3
# train_test_size = 0.33

In [ ]:
t0 = time.time()
errors_priv, errors_nonpriv= solve(X, y, alpha_parameter)
t1 = time.time()
print('Runtime : ' + str(t1-t0))

In [ ]:
errors_priv

In [ ]:
errors_nonpriv